# Add title
Add citation to published paper

In [1]:
import pandas as pd
import re
import pickle

## Import scraped data
Details about the collection of data from LibGuides is available in the full paper and the code used to scrape the original guides is available upon request (email chennesy@umn.edu).

The data used below is available in the Data Repository for U of M (DRUM). [add link]

In [2]:
#adjust the line below to point to the CSV file that you've downloaded from DRUM.
df_links = pd.read_csv('csvs_output/drum_lg_data.csv')

In [3]:
df_links.columns

Index(['resource_name_normalized', 'resource_name_scraped', 'resource_url',
       'resource_url_simple', 'resource_domain', 'guide_url',
       'institution_site_id'],
      dtype='object')

### Descriptive statistics

#### Number of resources
"Resources" refers to the links collected from all guides. They include "link assets" and "database assets" (according to the terms used in LibGuides) as well as any other links added in the WYSIWYG tool in LibGuides. 

In [4]:
print('Number of non-unique resources in dataset:', '{:,}'.format(len(df_links)), '\n')
print('Unique resource names',
      '\n----------------------------------------------------------------------',
      '\nUnique resource names, after initial cleaning but before normalization:', df_links['resource_name_scraped'].nunique(), 
      '\nUnique resource names after cleaning and normalization:', df_links['resource_name_normalized'].nunique(), 
     '\nResource names normalized through cleaning:', df_links['resource_name_scraped'].nunique() -  df_links['resource_name_normalized'].nunique(), '('
     '{:.2%}'.format((df_links['resource_name_scraped'].nunique() -  df_links['resource_name_normalized'].nunique())/ df_links['resource_name_scraped'].nunique())+')'
       '\n----------------------------------------------------------------------')

Number of non-unique resources in dataset: 186,952 

Unique resource names 
---------------------------------------------------------------------- 
Unique resource names, after initial cleaning but before normalization: 77094 
Unique resource names after cleaning and normalization: 64131 
Resource names normalized through cleaning: 12963 (16.81%)
----------------------------------------------------------------------


#### Number of guide pages

In [5]:
n_links_per_guide_page = df_links['guide_url'].value_counts()

n_guides = df_links['guide_url'].nunique()
print('Number of guide pages included in study:', n_guides)

Number of guide pages included in study: 10448


#### Resources per institution

In [6]:
links_per_site = df_links.groupby('institution_site_id')['resource_name_normalized'].count().sort_values(ascending=False)
unique_links_per_site = df_links.groupby('institution_site_id')['resource_name_normalized'].nunique().sort_values(ascending=False)

print('Number of institutions:',
      df_links['institution_site_id'].nunique(),
     '\n-------------------------------------------------------------\n')

for i, df in enumerate([links_per_site, unique_links_per_site]):
    if i == 0:
        print('Non-unique links per institution:')
    else:
        print('Unique links per institution:')
    print(
      '-------------------------------------------------------------',
      '\nMaximum:', df.max(),
      '\nMinimum:', df.min(),
      '\nMean:', '{:.2f}'.format(df.mean()), 
      '\nStandard deviation', '{:.2f}'.format(df.std()),
     '\n-------------------------------------------------------------')


Number of institutions: 123 
-------------------------------------------------------------

Non-unique links per institution:
------------------------------------------------------------- 
Maximum: 10799 
Minimum: 21 
Mean: 1519.93 
Standard deviation 1395.24 
-------------------------------------------------------------
Unique links per institution:
------------------------------------------------------------- 
Maximum: 3284 
Minimum: 21 
Mean: 791.96 
Standard deviation 638.44 
-------------------------------------------------------------


#### Guide pages per site

In [7]:
unique_pages_per_site = df_links.groupby('institution_site_id')['guide_url'].nunique().sort_values(ascending=False)

print('Guide pages per institution:')
print(
  '-------------------------------------------------------------',
  '\nMaximum:', unique_pages_per_site.max(),
  '\nMinimum:', unique_pages_per_site.min(),
  '\nMean:', '{:.2f}'.format(unique_pages_per_site.mean()), 
  '\nStandard deviation', '{:.2f}'.format(unique_pages_per_site.std()),
 '\n-------------------------------------------------------------')


Guide pages per institution:
------------------------------------------------------------- 
Maximum: 336 
Minimum: 1 
Mean: 84.94 
Standard deviation 64.15 
-------------------------------------------------------------


### Top 500 resources
Creates a link_site_df dataframe containing the 500 most common resources by the 'resource_name_normalized' column.

In [8]:
#save the top unique resource names to new dataframe
unique_names = df_links['resource_name_normalized'].value_counts()
top_names = unique_names.to_frame()

#unique site_ids to list
site_ids = df_links['institution_site_id'].unique().tolist()

#create empty df to populate with names and site_id counts and
#assign site_ids as column names
link_site_df = pd.DataFrame(columns=site_ids)

#add name column and add top 500 databases as values
link_site_df['name'] = top_names.index[0:500]

#move name column to beginning of df
cols = list(link_site_df.columns)
cols = [cols[-1]] + cols[:-1]
link_site_df = link_site_df[cols]

#set name column as index
link_site_df = link_site_df.set_index(['name'])

#add counts of each resource to df

#add value of 1 if name is found in site_id and 0 if name is not found for site_id
for site in site_ids:
    current_site = df_links[df_links['institution_site_id'] == site]
    for name in top_names.index[0:500]:
        
        matches = current_site[current_site['resource_name_normalized'] == name]
        if len(matches) >= 1:
            link_site_df.loc[name, site] = 1
        else:
            link_site_df.loc[name, site] = 0

#create columns for sum and percent of how many sites include each link name
link_site_df['sum'] = link_site_df.sum(axis=1)
link_site_df['percent'] = link_site_df['sum']/len(site_ids)
link_site_df['percent_of_sites'] = (100. * link_site_df['percent']).round(2).astype(str) + '%'
link_site_df = link_site_df.sort_values(by=['percent'], ascending=False)

#add column with full counts per db
link_site_df['count'] = ''

#add counts for top 500 links to df
for i,row in top_names[0:500].iterrows():
    name = i
    count = row['resource_name_normalized']
    link_site_df.loc[name, 'count'] = count

#add column with an example url that equals the most common (simplified) url found per resource name

#create empty column
link_site_df['url'] = ''

#for each link in top_names, create a subset dataframe and then count the most common simple urls
#then add the simple url next to the appropriate row in link_site_df

for i,row in top_names[0:500].iterrows():
    name = i
    current_site = df_links[df_links['resource_name_normalized'] == i]
    most_common_url = current_site['resource_url_simple'].value_counts()
    mc_url = most_common_url.index[0]
    link_site_df.loc[name, 'url'] = mc_url

link_site_df.reset_index(inplace=True)

At this stage the 'name', 'sum', 'percent_of_sites', 'count', and 'url' columns were exported to CSV and added to a Google Sheet where co-authors annotated the sheet to note whether each of the top resources was a 'free', 'paid', or 'hybrid' resource.

### Top 20 Resources (Sorted by percentage)
Table of the twenty resources most commonly included across institutional sites.

In [9]:
table_top_resources = link_site_df[['name','percent_of_sites', 'count', 'url']][0:20]
table_top_resources = table_top_resources.rename(columns ={'name': 'Resource name', 'percent_of_sites':'Percentage of institutions', 'count':'Overall count', 'url':'Example URL'})
table_top_resources

,Resource name,Percentage of institutions,Overall count,Example URL
0,icpsr,94.31%,1834,www.icpsr.umich.edu
1,data.gov,94.31%,954,www.data.gov
2,u.s. census bureau,93.5%,956,www.census.gov
3,u.s. census: data,90.24%,1145,data.census.gov/cedsci
4,undata (united nations),88.62%,1054,data.un.org
5,nces (u.s. national center for education stati...,86.99%,546,nces.ed.gov
6,nchs (u.s. national center for health statistics),86.99%,856,www.cdc.gov/nchs
7,u.s. census: american factfinder,83.74%,720,factfinder.census.gov/faces/nav/jsf/pages/inde...
8,bls (u.s. bureau of labor statistics),82.93%,535,www.bls.gov
9,roper center: ipoll,80.49%,1090,proxy2.library.illinois.edu/login?url=ropercen...


### Top 10 Resources (Sorted by overall count)

In [10]:
top_10 = link_site_df[['name','percent_of_sites', 'count', 'url']].sort_values(by='count', ascending=False)[0:10]
top_10

,name,percent_of_sites,count,url
0,icpsr,94.31%,1834,www.icpsr.umich.edu
31,statistical insight (proquest),61.79%,1532,proxy2.library.illinois.edu/login?url=search.p...
12,social explorer,78.86%,1430,www.socialexplorer.com
51,data planet (sage),50.41%,1318,go.openathens.net/redirector/fiu.edu?url=datap...
3,u.s. census: data,90.24%,1145,data.census.gov/cedsci
28,statista,64.23%,1104,go.openathens.net/redirector/fiu.edu?url=http%...
9,roper center: ipoll,80.49%,1090,proxy2.library.illinois.edu/login?url=ropercen...
13,statistical abstract of the united states (pro...,78.05%,1058,www.libraries.rutgers.edu/indexes/statabus
4,undata (united nations),88.62%,1054,data.un.org
2,u.s. census bureau,93.5%,956,www.census.gov


### Top 10 (full) Domains

In [11]:
unique_domain_counts = df_links['resource_domain'].value_counts()
domains_top = unique_domain_counts.to_frame()
domains_top = domains_top.reset_index()
domains_top = domains_top.rename(columns = {'index': 'Domain', 'domain': 'Count'})
domains_top[0:10]

,Domain,resource_domain
0,census.gov,7656
1,cdc.gov,5382
2,nces.ed.gov,2546
3,icpsr.umich.edu,2222
4,bls.gov,1928
5,go.openathens.net,1837
6,who.int,1202
7,utk.primo.exlibrisgroup.com,1128
8,data.worldbank.org,1120
9,nrs.harvard.edu,1120


### Top 20 Root Domains
Note that root domains such as harvard.edu, exlibrisgroup.com, upenn.edu, cornell.edu, openathens.net, and msu.edu are not included in the final paper Table since the most common resources included from those domains were not relevant to our analysis. See full output below for example subdomains for each.

In [12]:
#separate the top level domain (this is slightly coarse but works for the top sites)
df_links['top_level_domain'] = df_links.resource_domain.str.split(r'([^.\s]+\.[^.\s]+)$').str[1]
top_level_domain_counts = df_links['top_level_domain'].value_counts()

root_domains_top = top_level_domain_counts.to_frame()
root_domains_top = root_domains_top.reset_index()
root_domains_top = root_domains_top.rename(columns = {'index': 'Root domain', 'top_level_domain': 'Count'})
root_domains_top[0:20]

,Root domain,Count
0,census.gov,9670
1,cdc.gov,6072
2,harvard.edu,4480
3,umich.edu,3691
4,ed.gov,3122
5,exlibrisgroup.com,2744
6,worldbank.org,2705
7,un.org,2636
8,upenn.edu,2319
9,cornell.edu,2155


#### Print the ten most common domains for each of the top 20 root domains

In [13]:
root_domains_20 = {}
for i, row in root_domains_top[0:50].iterrows():
    root_domains_20[row['Root domain']] = row['Count']

for k,v in root_domains_20.items():
    domain_check = domains_top[domains_top['Domain'].str.contains(k)]
    print(k, '\n------------------------------------------------\n',
          k, 'total count:', v, '\n', 
          k, '# of domains:', len(domain_check), '\n',
          k, 'mean count per domain:', '{:.2f}'.format(v/len(domain_check)), '\n\n', 
          domain_check[0:10],
         '\n------------------------------------------------\n')

census.gov 
------------------------------------------------
 census.gov total count: 9670 
 census.gov # of domains: 42 
 census.gov mean count per domain: 230.24 

                       Domain  resource_domain
0                 census.gov             7656
14           data.census.gov              945
63     factfinder.census.gov              343
75    factfinder2.census.gov              316
467   dataferrett.census.gov               61
623      usatrade.census.gov               45
678      lehd.ces.census.gov               41
690    quickfacts.census.gov               41
819           ask.census.gov               33
962  onthemap.ces.census.gov               27 
------------------------------------------------

cdc.gov 
------------------------------------------------
 cdc.gov total count: 6072 
 cdc.gov # of domains: 26 
 cdc.gov mean count per domain: 233.54 

                              Domain  resource_domain
1                           cdc.gov             5382
65             

### Top 20 Free and Paid/Hybrid Resources
The top_dbs_annotated.csv file was created by adding an 'access' column noting whether each of the top 500 resources identified and exported above is 'free', 'paid', or 'hybrid'.

In [36]:
free_paid_df = pd.read_csv('csvs_input/top_dbs_annotated.csv', index_col='Unnamed: 0')
free_paid_df = free_paid_df.sort_values(by='count', ascending=False)
free_paid_df = free_paid_df.drop(columns=['sum'])

paid_df = free_paid_df[free_paid_df['access'] == 'paid']
free_df = free_paid_df[free_paid_df['access'] == 'free']
hybrid_df = free_paid_df[free_paid_df['access'] == 'hybrid']
paid_hybrid_df = free_paid_df[free_paid_df['access'] != 'free']

print(
    'Percentage of top 500 resources that are paid or hybrid:', 
    '{:.2%}'.format((len(paid_hybrid_df)/len(free_paid_df))),
    '\nNumber that are paid or hybrid:', 
    len(paid_hybrid_df),
    '\n\tPercentage of top 500 resources that are paid:', 
    '{:.2%}'.format((len(paid_df)/len(free_paid_df))),
    '\n\tNumber that are paid:', 
    len(paid_df),
    '\n\tPercentage of top 500 resources that are hybrid:', 
    '{:.2%}'.format((len(hybrid_df)/len(free_paid_df))),
    '\n\tNumber that are hybrid:', 
    len(hybrid_df),
     '\nPercentage of top 500 resources that are free:',               
      '{:.2%}'.format((len(free_df)/len(free_paid_df))), 
    '\nNumber that are free:', 
    len(free_df),
                   )

print(
    '\n\nNumber of links for paid/hybrid resources:', paid_hybrid_df['count'].sum(),
    '\nMean number of links per paid/hybrid resource:', '{:.2f}'.format(paid_hybrid_df['count'].mean()),
    '\n\n\tNumer of links to paid resources:', paid_df['count'].sum(),
    '\n\tMean number of links per paid resource:', '{:.2f}'.format(paid_df['count'].mean()),
    '\n\tMedian number of links per paid resource:', '{:.2f}'.format(paid_df['count'].median()),
    '\n\n\tNumer of links to hybrid resources:', hybrid_df['count'].sum(),
    '\n\tMean number of links per hybrid resource:', '{:.2f}'.format(hybrid_df['count'].mean()),
    '\n\tMedian number of links per hybrid resource:', '{:.2f}'.format(hybrid_df['count'].median()),
    '\n\nNumber of links for free resources:', free_df['count'].sum(),
    '\nMean number of links per free resource:', '{:.2f}'.format(free_df['count'].mean()),
    '\nMedian number of links per free resource:', '{:.2f}'.format(free_df['count'].median())
)

Percentage of top 500 resources that are paid or hybrid: 20.28% 
Number that are paid or hybrid: 101 
	Percentage of top 500 resources that are paid: 17.47% 
	Number that are paid: 87 
	Percentage of top 500 resources that are hybrid: 2.81% 
	Number that are hybrid: 14 
Percentage of top 500 resources that are free: 79.72% 
Number that are free: 397


Number of links for paid/hybrid resources: 19627 
Mean number of links per paid/hybrid resource: 194.33 

	Numer of links to paid resources: 15175 
	Mean number of links per paid resource: 174.43 
	Median number of links per paid resource: 66.00 

	Numer of links to hybrid resources: 4452 
	Mean number of links per hybrid resource: 318.00 
	Median number of links per hybrid resource: 67.50 

Number of links for free resources: 41361 
Mean number of links per free resource: 104.18 
Median number of links per free resource: 53.00


In [28]:
paid_hybrid_df[0:20]

,name,percent_of_sites,count,url,access
1,icpsr,94.31%,1834,www.icpsr.umich.edu,hybrid
34,statistical insight (proquest),61.79%,1532,proxy2.library.illinois.edu/login?url=search.p...,paid
11,social explorer,78.86%,1430,www.socialexplorer.com,paid
52,data planet (sage),50.41%,1318,go.openathens.net/redirector/fiu.edu?url=datap...,paid
29,statista,64.23%,1104,go.openathens.net/redirector/fiu.edu?url=http%...,hybrid
20,roper center: ipoll,80.49%,1090,proxy2.library.illinois.edu/login?url=ropercen...,paid
21,statistical abstract of the united states (pro...,78.05%,1058,www.libraries.rutgers.edu/indexes/statabus,paid
23,oecd ilibrary,69.11%,851,www.oecd-ilibrary.org,hybrid
15,historical statistics of the united states (ca...,74.80%,684,hsus.cambridge.org/HSUSWeb,paid
67,simplyanalytics,44.72%,672,go.openathens.net/redirector/fiu.edu?url=app.s...,paid


In [29]:
free_df[0:20]

,name,percent_of_sites,count,url,access
4,u.s. census: data,90.24%,1145,data.census.gov/cedsci,free
5,undata (united nations),88.62%,1054,data.un.org,free
3,u.s. census bureau,93.50%,956,www.census.gov,free
2,data.gov,94.31%,954,www.data.gov,free
7,nchs (u.s. national center for health statistics),86.99%,856,www.cdc.gov/nchs,free
8,u.s. census: american factfinder,83.74%,720,factfinder.census.gov/faces/nav/jsf/pages/inde...,free
26,world bank: world development indicators,65.85%,599,data.worldbank.org/data-catalog/world-developm...,free
6,nces (u.s. national center for education stati...,86.99%,546,nces.ed.gov,free
14,world bank: data,76.42%,545,data.worldbank.org,free
9,bls (u.s. bureau of labor statistics),82.93%,535,www.bls.gov,free


In [51]:
hybrid_df[0:20]

,name,percent_of_sites,count,url,access
1,icpsr,94.31%,1834,www.icpsr.umich.edu,hybrid
29,statista,64.23%,1104,go.openathens.net/redirector/fiu.edu?url=http%...,hybrid
23,oecd ilibrary,69.11%,851,www.oecd-ilibrary.org,hybrid
198,oecd ilibrary: statistics,22.76%,110,www.oecd-ilibrary.org/statistics,hybrid
449,rand state statistics,10.57%,83,www.randstatestats.org,hybrid
132,oecd (organisation for economic co-operation a...,30.08%,78,www.oecd.org,hybrid
173,pollingreport.com,23.58%,68,www.pollingreport.com,hybrid
187,icpsr: health and medical care archive (hmca),22.76%,67,www.icpsr.umich.edu/web/pages/HMCA/index.html,hybrid
450,wolframalpha,10.57%,58,www.wolframalpha.com,hybrid
500,byu law and corpus linguistics,0.81%,55,lawcorpus.byu.edu,hybrid


#### Output the Top 200 resources (annotated as free/paid/hybrid) to HTML Table

In [49]:
for_html = free_paid_df.sort_values(by='count', ascending=False)[0:200]

for_html = for_html.reset_index()
for_html.index += 1 
for_html = for_html[['name', 'percent_of_sites', 'count', 'url', 'access']]

html = for_html.to_html()
  
# write html to file
text_file = open("table.html", "w")
text_file.write(html)
text_file.close()